In [15]:
!pip install pymysql

In [16]:
from time import sleep
from threading import *
import shutil
import os
import warnings
import pymysql
warnings.filterwarnings('ignore')

In [17]:
class Folders:
    def __init__(self):
        self.Queue = "Queue"
        self.Processing = "Processing"
        self.Processed = "Processed"
        con=pymysql.connect(host='localhost',user='root',password='root',database='assignment')
        cur=con.cursor()
        cur.execute("delete from files")
        con.commit()
        con.close()

        if(os.path.exists(self.Queue)) and (os.path.exists(self.Processed)) and (os.path.exists(self.Processing)):
            shutil.rmtree(self.Queue)
            shutil.rmtree(self.Processing)
            shutil.rmtree(self.Processed)
            
            os.mkdir(self.Queue)
            os.mkdir(self.Processing)
            os.mkdir(self.Processed)
        else:
            os.mkdir(self.Queue)
            os.mkdir(self.Processing)
            os.mkdir(self.Processed)

In [18]:
class Processing(Thread):
    def run(self):
        i = 1
        while(True):
            open("Processing/file_"+str(i)+".txt","w")
            i += 1
            sleep(1)

In [19]:
class Queue(Thread):
    def run(self):
        src = 'Processing'
        dest = 'Queue'
        while(True):
            if len(os.listdir(src)) == 5: 
                for file in os.listdir(src):
                    file_name = os.path.join(src, file)
                    shutil.move(file_name, dest)
                    

In [20]:
class Processed(Thread):
    def run(self):
        dest = 'Processed'
        src = 'Queue'
       
        while(True):
            sleep(2)
            if len(os.listdir(src)) != 0:
                for file in os.listdir(src):
                    con=pymysql.connect(host='localhost',user='root',password='root',database='assignment')
                    cur=con.cursor()
                    cur.execute('insert into files values(%s,%s)',(file,1))
                    con.commit()
                    con.close()
                    file_name = os.path.join(src, file)
                    shutil.move(file_name, dest)

In [21]:
f = Folders()
Processing_thread = Processing()
Queue_thread = Queue()
Processed_thread = Processed()
Processing_thread.start()
Queue_thread.start()
Processed_thread.start()

OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] No connection could be made because the target machine actively refused it)")